In [1]:
import os
import torch
import numpy as np
import csv
from torchvision import models, transforms
from PIL import Image
from scipy.linalg import sqrtm

# Load the pretrained Inception model
inception = models.inception_v3(pretrained=True, transform_input=True).eval()

# Define the transformation to resize and normalize images
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_features(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        features = inception(image)

    return features.squeeze().numpy()

def calculate_fid(features1, features2):
    features1 = features1.reshape(1, -1) if features1.ndim == 1 else features1
    features2 = features2.reshape(1, -1) if features2.ndim == 1 else features2

    mu1, sigma1 = features1.mean(axis=0), np.cov(features1, rowvar=False)
    mu2, sigma2 = features2.mean(axis=0), np.cov(features2, rowvar=False)

    if sigma1.size == 1:
        sigma1 = np.zeros((features1.shape[1], features1.shape[1]))
    if sigma2.size == 1:
        sigma2 = np.zeros((features2.shape[1], features2.shape[1]))

    diff = mu1 - mu2
    covmean = sqrtm(sigma1 @ sigma2)

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid_score = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid_score

def evaluate_fid_for_all_batches(path_generated, path_labels, output_csv):
    # Explicit path creation for debugging
    output_csv = os.path.abspath(output_csv)  ### Ensure full path for debugging
    print(f"Attempting to write CSV to: {output_csv}")  ### Debugging path print

    fid_scores = []
    label_epoch = 1

    # Check if we have permission to write in this directory
    try:
        with open(output_csv, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Epoch', 'Batch', 'FID Score'])  # Header row
            print(f"CSV file created successfully: {output_csv}")  ### Confirmation of file creation

            generated_files = os.listdir(path_generated)
            epochs = set()
            batches = set()
            for filename in generated_files:
                if filename.startswith('y_gen_') and filename.endswith('.tiff'):
                    parts = filename.split('_')
                    epoch_num = int(parts[2])
                    batch_num = int(parts[4].split('.')[0])
                    epochs.add(epoch_num)
                    batches.add(batch_num)

            epochs = sorted(epochs)
            batches = sorted(batches)

            for epoch in epochs:
                for batch in batches:
                    gen_image_path = os.path.join(path_generated, f'y_gen_{epoch}_batch_{batch}.tiff')
                    label_image_path = os.path.join(path_labels, f'label_{label_epoch}_batch_{batch}.tiff')

                    if os.path.exists(gen_image_path) and os.path.exists(label_image_path):
                        features_gen = get_features(gen_image_path)
                        features_label = get_features(label_image_path)

                        fid_score = calculate_fid(features_gen, features_label)
                        writer.writerow([epoch, batch, fid_score])  # Write to CSV
                        file.flush()
                        print(f"Saved to CSV: Epoch {epoch}, Batch {batch}, FID Score = {fid_score}")
                    else:
                        print(f"Missing files for Epoch {epoch}, Batch {batch}. Skipping...")

    except IOError as e:
        print(f"Error creating or writing to the file: {e}")  ### Error message for file creation failure

# Example usage
path_generated = '/home/dt681254/jupyterlab/CGAN/evaluation'  # Replace with the path to your generated images
path_labels = '/home/dt681254/jupyterlab/CGAN/evaluation'         # Replace with the path to your label images
output_csv = '/rwthfs/rz/cluster/home/dt681254/jupyterlab/CGAN/Evaluierung/fid_scores.csv'        ### Ensure this path is writable and correct

evaluate_fid_for_all_batches(path_generated, path_labels, output_csv)
print(f"All FID scores saved to {output_csv}")

/home/dt681254/.conda/envs/GAN/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dt681254/.conda/envs/GAN/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Attempting to write CSV to: /rwthfs/rz/cluster/home/dt681254/jupyterlab/CGAN/Evaluierung/fid_scores.csv
CSV file created successfully: /rwthfs/rz/cluster/home/dt681254/jupyterlab/CGAN/Evaluierung/fid_scores.csv
Saved to CSV: Epoch 0, Batch 0, FID Score = 1341.22265625
Saved to CSV: Epoch 0, Batch 1, FID Score = 1603.31982421875
Saved to CSV: Epoch 0, Batch 2, FID Score = 1592.07568359375
Saved to CSV: Epoch 0, Batch 3, FID Score = 1339.2354736328125
Saved to CSV: Epoch 0, Batch 4, FID Score = 1372.544921875
Saved to CSV: Epoch 0, Batch 5, FID Score = 1183.6298828125
Saved to CSV: Epoch 0, Batch 6, FID Score = 1504.556884765625
Saved to CSV: Epoch 0, Batch 7, FID Score = 1611.381103515625
Saved to CSV: Epoch 0, Batch 8, FID Score = 1396.8277587890625
Saved to CSV: Epoch 0, Batch 9, FID Score = 1668.233154296875
Saved to CSV: Epoch 0, Batch 10, FID Score = 1583.65478515625
Saved to CSV: Epoch 0, Batch 11, FID Score = 1391.0008544921875
Saved to CSV: Epoch 0, Batch 12, FID Score = 1837.84